In [1]:
import pandas as pd
import numpy as np
import pickle
import nltk
import re
import string
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy
from collections import Counter
import scipy.sparse as ss
from sklearn import datasets
from corextopic import corextopic as ct
from corextopic import vis_topic as vt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Read in the transcripts
with open('transcripts.pickle','rb') as read_file:
    transcripts = pickle.load(read_file)

## Create df of 5mins of text/ row

In [ ]:
def bin_5min_blocks(df, key):
    text_list = []
    time_list = []
    key_list = []
    i=0
    while i < len(df):
        current_time = df.start.iloc[i]
        current_text = ''
        while (i < len(df)) and (df.start.iloc[i] <= current_time+300):
            current_text = current_text + ' ' + df.text.iloc[i]
            i +=1
        time_list.append(current_time)
        text_list.append(current_text)
        key_list.append(key)
    return pd.DataFrame(list(zip(key_list, time_list, text_list)),\
                        columns=['episode', 'time','text'])

In [ ]:
binned_df = pd.DataFrame(columns=['episode','time','text'])
for key in transcripts.keys():
    binned_df = binned_df.append(bin_5min_blocks(pd.DataFrame(transcripts[key]), key))

In [ ]:
binned_df.shape

In [ ]:
# Pickle the 5 min bin df
with open('binned_df.pickle', 'wb') as to_write:
    pickle.dump(binned_df, to_write)

# spaCy

In [ ]:
# Read in the bin df
with open('binned_df.pickle','rb') as read_file:
    binned_df = pickle.load(read_file)

binned_df = binned_df.reset_index().drop(columns='index')

In [ ]:
# Load spaCy's english core module

nlp = spacy.load('en_core_web_sm')

### Tokenize

In [ ]:
# Tokenization (takes 5-10 min to run)

binned_df['spacy_doc'] = list(nlp.pipe(binned_df.text))

# to tokenize one string:
# doc = nlp(text)

In [ ]:
# Pickle the 5 min bin tokenized text (1.59 GB)
with open('token_5min_df.pickle', 'wb') as to_write:
    pickle.dump(binned_df, to_write)

In [3]:
# Read in the 5 min tokenized text
with open('token_5min_df.pickle','rb') as read_file:
    binned_df = pickle.load(read_file)

### Put Nouns and Adjectives in a string for each doc

In [4]:
def key_words(row):
    current_string = ''
    for token in row['spacy_doc']:
        if (token.is_stop == False) and ((token.pos_ == 'NOUN') or (token.pos_ == 'ADJ')):
            current_word = token.lemma_
            current_string = current_string + current_word + ' '
    return current_string

In [5]:
binned_df['key_words'] = binned_df.apply(key_words ,axis=1)

# Vectorize

In [6]:
vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df=0.3)
dtm = vectorizer.fit_transform(binned_df.key_words)

# Fit LDA Model

In [7]:
lda = LatentDirichletAllocation(n_components=16, random_state=0)
doc_topic = lda.fit_transform(dtm)

### Pickle Model, dtm and vectorizer

In [8]:
# Pickle the LDA Model, dtm, and vectorizer

mmv = [lda, dtm, vectorizer, doc_topic]

with open('mmv_5min.pickle', 'wb') as to_write:
    pickle.dump(mmv, to_write)

In [9]:
# Read in the LDA MOdel
with open('mmv_5min.pickle','rb') as read_file:
    mmv = pickle.load(read_file)

lda = mmv[0]
dtm = mmv[1]
vectorizer = mmv[2]
doc_topic = mmv[3]

### pyLDAvis

In [10]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, dtm, vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.049489  0.096537       1        1  11.718838
4      0.056653  0.042013       2        1  11.386334
6     -0.013507  0.261311       3        1  11.223670
9      0.142905  0.197054       4        1   8.621766
7     -0.204217  0.209423       5        1   8.245150
12     0.053294 -0.260096       6        1   7.968779
0      0.050422  0.219747       7        1   7.722403
1      0.003453 -0.109530       8        1   7.353923
3     -0.302578  0.097065       9        1   6.913266
11     0.282869  0.109834      10        1   3.939080
15     0.123931 -0.091093      11        1   3.927340
13    -0.264628 -0.028599      12        1   3.056620
10     0.309462 -0.064245      13        1   2.298855
5     -0.250646 -0.171119      14        1   1.961786
14    -0.138515 -0.262489      15        1   1.861484
8      0.200591 -0.245812      16        1   1.800705, topic_info=            Term         Freq        Total Category  logprob  loglift
12489       tire  1013.000000  1013.000000  Default  30.0000  30.0000
12236       team  1750.000000  1750.000000  Default  29.0000  29.0000
7901      muscle  1827.000000  1827.000000  Default  28.0000  28.0000
12335       test   997.000000   997.000000  Default  27.0000  27.0000
12643    trainer  1405.000000  1405.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
13654      wheel    32.791024  1023.945485  Topic16  -5.3591   0.5757
11088  situation    31.015143   789.856057  Topic16  -5.4148   0.7796
8582        pace    28.744872   567.166475  Topic16  -5.4908   1.0348
6980        life    28.559141   750.976758  Topic16  -5.4973   0.7476
7606        mile    28.932040  1079.472789  Topic16  -5.4843   0.3977

[1164 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
45         3  0.712150  acceleration
45         4  0.173695  acceleration
45        10  0.112902  acceleration
52         1  0.024772        access
52         2  0.041286        access
...      ...       ...           ...
13891      6  0.089597       youtube
13891      8  0.689894       youtube
13891     11  0.017919       youtube
13918      6  0.958790        zipper
13921     15  0.808735        zombie

[5076 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 7, 10, 8, 13, 1, 2, 4, 12, 16, 14, 11, 6, 15, 9])

In [11]:
# Get top words for each  topic
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

/Users/Jay/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
display_topics(lda, vectorizer.get_feature_names(), 16)


Topic  0
weight, strength, muscle, pound, fat, leg, fiber, strong, endurance, calorie, cyclist, type, watt, mass, gym, force

Topic  1
coach, thank, cycling, awesome, trainerroad, group, episode, live, trainer, video, cyclist, team, fun, new, forum, pro

Topic  2
plan, volume, recovery, stress, base, fitness, season, event, intensity, sweet, life, important, phase, month, rest, build

Topic  3
food, bottle, calorie, gram, carbohydrate, water, nutrition, fat, gel, station, carb, caffeine, protein, morning, aid, meal

Topic  4
mountain, climb, mile, cross, section, flat, trail, skill, track, technical, effort, group, speed, rider, stage, fun

Topic  5
test, ramp, collagen, breath, breathing, air, cold, allergy, mouth, nose, asthma, nitrate, interesting, meat, result, number

Topic  6
team, rider, group, field, line, lap, wheel, sprint, turn, breakaway, corner, crit, wind, speed, sprinter, attack

Topic  7
muscle, study, blood, cell, effect, performance, exercise, fat, endurance, brain, 

/Users/Jay/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# make weight readable
topic_list = ['strength','tr_chat','training_plan_structure','nutrition',\
              'mtb_racing','testing_medical','road_racing','physiology',\
              'interuptions','workout_structure','bike_mechanics','trainer_technical',\
              'riding_technical','heat_sweat','special_physical_cases','goals']
doc_topic_df = pd.DataFrame(doc_topic, columns = topic_list)
doc_topic_df = pd.concat([binned_df[['episode','time']], doc_topic_df], axis=1)

/Users/Jay/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
